In [1]:
import os
import mindspore
from mindnlp.transformers import AutoModelForSeq2SeqLM
from mindnlp.peft import get_peft_model, LoraConfig, TaskType
from mindnlp.dataset import load_dataset
from mindnlp.core import ops

from mindnlp.transformers import AutoTokenizer
from mindnlp.common.optimization import get_linear_schedule_with_warmup
from tqdm import tqdm

model_name_or_path = "lucadiliello/bart-small"
tokenizer_name_or_path = "lucadiliello/bart-small"

checkpoint_name = "financial_sentiment_analysis_lora_v1.ckpt"
max_length = 128
lr = 1e-3
num_epochs = 5
batch_size = 8

/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/ma-user/anaconda3/envs/MindSpore/lib

In [2]:
# creating model
peft_config = LoraConfig(task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1)

model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.
BartForConditionalGeneration has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`.`PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


[MS_ALLOC_CONF]Runtime config:  enable_vmm:True  vmm_align_size:2MB
trainable params: 294,912 || all params: 70,697,472 || trainable%: 0.4171464575140678


In [3]:
model

PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): BartForConditionalGeneration(
      (model): BartModel(
        (shared): Embedding(50265, 512, padding_idx=1)
        (encoder): BartEncoder(
          (embed_tokens): BartScaledWordEmbedding(50265, 512, padding_idx=1)
          (embed_positions): BartLearnedPositionalEmbedding(514, 512)
          (layers): ModuleList(
            (0-5): 6 x BartEncoderLayer(
              (self_attn): BartAttention(
                (k_proj): Linear (512 -> 512)
                (v_proj): lora.Linear(
                  (base_layer): Linear (512 -> 512)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.1, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear (512 -> 8)
                  )
                  (lora_B): ModuleDict(
                    (default): Linear (8 -> 512)
                  )
                  (lora_embedding_A): ParameterDic

In [4]:
mindspore.dataset.config.set_seed(0)
# loading dataset
dataset = load_dataset("financial_phrasebank", "sentences_allagree",cache_dir='/home/ma-user/work/financial_phrasebank/')

In [5]:
classes = dataset.source.ds.features["label"].names
classes

['negative', 'neutral', 'positive']

In [6]:
train_dataset, validation_dataset = dataset.shuffle(64).split([0.9, 0.1])

[WARNING] ME(236609:281473119305904,MainProcess):2024-12-07-14:56:06.521.579 [mindspore/dataset/engine/datasets.py:1217] Dataset is shuffled before split.


In [7]:
def add_text_label(sentence, label):
    return sentence, label, classes[label.item()]

train_dataset = train_dataset.map(add_text_label, ['sentence', 'label'], ['sentence', 'label', 'text_label'])
validation_dataset = validation_dataset.map(add_text_label, ['sentence', 'label'], ['sentence', 'label', 'text_label'])

In [8]:
next(train_dataset.create_dict_iterator())

{'sentence': Tensor(shape=[], dtype=String, value= '`` Serving customers locally is one of the cornerstones of customer focus for us .'),
 'label': Tensor(shape=[], dtype=Int64, value= 1),
 'text_label': Tensor(shape=[], dtype=String, value= 'neutral')}

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/mindnlp/transformers/tokenization_utils_base.py:1526: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted, and will be then set to `False` by default. 
  warnings.warn(


In [10]:
import numpy as np
from mindnlp.dataset import BaseMapFunction
from threading import Lock
lock = Lock()

class MapFunc(BaseMapFunction):
    def __call__(self, sentence, label, text_label):
        lock.acquire()
        model_inputs = tokenizer(sentence, max_length=max_length, padding="max_length", truncation=True)
        labels = tokenizer(text_label, max_length=3, padding="max_length", truncation=True)
        lock.release()
        labels = labels['input_ids']
        labels = np.where(np.equal(labels, tokenizer.pad_token_id), -100, labels)
        return model_inputs['input_ids'], model_inputs['attention_mask'], labels


def get_dataset(dataset, tokenizer, shuffle=True):
    input_colums=['sentence', 'label', 'text_label']
    output_columns=['input_ids', 'attention_mask', 'labels']
    dataset = dataset.map(MapFunc(input_colums, output_columns),
                          input_colums, output_columns)
    if shuffle:
        dataset = dataset.shuffle(64)
    dataset = dataset.batch(batch_size)
    return dataset

train_dataset = get_dataset(train_dataset, tokenizer)
eval_dataset = get_dataset(validation_dataset, tokenizer, shuffle=False)

In [11]:
next(train_dataset.create_dict_iterator())

{'input_ids': Tensor(shape=[8, 128], dtype=Int64, value=
 [[    0,   250,   367 ...     1,     1,     1],
  [    0, 20420,  1295 ...     1,     1,     1],
  [    0, 43623,     9 ...     1,     1,     1],
  ...
  [    0, 46769,  5516 ...     1,     1,     1],
  [    0,   133,   458 ...     1,     1,     1],
  [    0,   133,  1799 ...     1,     1,     1]]),
 'attention_mask': Tensor(shape=[8, 128], dtype=Int64, value=
 [[1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  ...
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0]]),
 'labels': Tensor(shape=[8, 3], dtype=Int64, value=
 [[    0, 12516,     2],
  [    0, 33407,     2],
  [    0, 12516,     2],
  ...
  [    0, 12516,     2],
  [    0, 12516,     2],
  [    0, 12516,     2]])}

In [12]:
from mindnlp.core import optim
# optimizer and lr scheduler
optimizer = optim.AdamW(model.trainable_params(), lr=lr)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataset) * num_epochs),
)

In [13]:
from mindnlp.core import value_and_grad
# training and evaluation
def forward_fn(**batch):
    outputs = model(**batch)
    loss = outputs.loss
    return loss

grad_fn = value_and_grad(forward_fn, model.trainable_params())

for epoch in range(num_epochs):
    model.set_train()
    total_loss = 0
    train_total_size = train_dataset.get_dataset_size()
    for step, batch in enumerate(tqdm(train_dataset.create_dict_iterator(), total=train_total_size)):
        optimizer.zero_grad()
        loss = grad_fn(**batch)
        optimizer.step()
        total_loss += loss.float()
        lr_scheduler.step()

    model.set_train(False)
    eval_loss = 0
    eval_preds = []
    eval_total_size = eval_dataset.get_dataset_size()
    for step, batch in enumerate(tqdm(eval_dataset.create_dict_iterator(), total=eval_total_size)):
        with mindspore._no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        eval_loss += loss.float()
        eval_preds.extend(
            tokenizer.batch_decode(ops.argmax(outputs.logits, -1).asnumpy(), skip_special_tokens=True)
        )

    eval_epoch_loss = eval_loss / len(eval_dataset)
    eval_ppl = ops.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / len(train_dataset)
    train_ppl = ops.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")

100%|██████████| 29/29 [00:02<00:00, 11.01it/s]


epoch=0: train_ppl=Tensor(shape=[], dtype=Float32, value= 1.37726) train_epoch_loss=Tensor(shape=[], dtype=Float32, value= 0.320096) eval_ppl=Tensor(shape=[], dtype=Float32, value= 1.10493) eval_epoch_loss=Tensor(shape=[], dtype=Float32, value= 0.0997784)


100%|██████████| 29/29 [00:02<00:00, 11.92it/s]


epoch=1: train_ppl=Tensor(shape=[], dtype=Float32, value= 1.08045) train_epoch_loss=Tensor(shape=[], dtype=Float32, value= 0.0773764) eval_ppl=Tensor(shape=[], dtype=Float32, value= 1.08047) eval_epoch_loss=Tensor(shape=[], dtype=Float32, value= 0.0774005)


100%|██████████| 29/29 [00:02<00:00, 12.04it/s]


epoch=2: train_ppl=Tensor(shape=[], dtype=Float32, value= 1.05117) train_epoch_loss=Tensor(shape=[], dtype=Float32, value= 0.0499013) eval_ppl=Tensor(shape=[], dtype=Float32, value= 1.10261) eval_epoch_loss=Tensor(shape=[], dtype=Float32, value= 0.0976802)


100%|██████████| 29/29 [00:02<00:00, 12.12it/s]


epoch=3: train_ppl=Tensor(shape=[], dtype=Float32, value= 1.04519) train_epoch_loss=Tensor(shape=[], dtype=Float32, value= 0.0441984) eval_ppl=Tensor(shape=[], dtype=Float32, value= 1.09936) eval_epoch_loss=Tensor(shape=[], dtype=Float32, value= 0.094732)


100%|██████████| 29/29 [00:02<00:00, 11.93it/s]

epoch=4: train_ppl=Tensor(shape=[], dtype=Float32, value= 1.02586) train_epoch_loss=Tensor(shape=[], dtype=Float32, value= 0.0255352) eval_ppl=Tensor(shape=[], dtype=Float32, value= 1.09519) eval_epoch_loss=Tensor(shape=[], dtype=Float32, value= 0.0909286)


In [14]:
# print accuracy
correct = 0
total = 0

ground_truth = []

for pred, data in zip(eval_preds, validation_dataset.create_dict_iterator(output_numpy=True)):
    true = str(data['text_label'])
    ground_truth.append(true)
    if pred.strip() == true.strip():
        correct += 1
    total += 1
accuracy = correct / total * 100
print(f"{accuracy=} % on the evaluation dataset")
print(f"{eval_preds[:10]=}")
print(f"{ground_truth[:10]=}")

accuracy=92.03539823008849 % on the evaluation dataset
eval_preds[:10]=['neutral', 'positive', 'neutral', 'neutral', 'neutral', 'neutral', 'negative', 'neutral', 'neutral', 'positive']
ground_truth[:10]=['neutral', 'positive', 'neutral', 'neutral', 'neutral', 'neutral', 'negative', 'neutral', 'neutral', 'positive']


In [15]:
next(eval_dataset.create_tuple_iterator())

[Tensor(shape=[8, 128], dtype=Int64, value=
 [[    0, 49519, 45338 ...     1,     1,     1],
  [    0,   597,  5246 ...     1,     1,     1],
  [    0,   133,  1110 ...     1,     1,     1],
  ...
  [    0,   133,   613 ...     1,     1,     1],
  [    0,   104,  3914 ...     1,     1,     1],
  [    0, 35028,  2156 ...     1,     1,     1]]),
 Tensor(shape=[8, 128], dtype=Int64, value=
 [[1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  ...
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0]]),
 Tensor(shape=[8, 3], dtype=Int64, value=
 [[    0, 12516,     2],
  [    0, 22173,     2],
  [    0, 12516,     2],
  ...
  [    0, 12516,     2],
  [    0, 33407,     2],
  [    0, 12516,     2]])]

In [16]:
# saving model
peft_model_id = f"{model_name_or_path}_{peft_config.peft_type}_{peft_config.task_type}"
model.save_pretrained(peft_model_id)

In [17]:
ckpt = f"{peft_model_id}/adapter_model.ckpt"
!du -h $ckpt

1.2M	lucadiliello/bart-small_LORA_SEQ_2_SEQ_LM/adapter_model.ckpt


In [18]:
from mindnlp.peft import PeftModel, PeftConfig

peft_model_id = f"{model_name_or_path}_{peft_config.peft_type}_{peft_config.task_type}"

config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(model, peft_model_id)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.


In [19]:
model.set_train(False)
example = next(validation_dataset.create_dict_iterator(output_numpy=True))
print("input",example['sentence'])
print("true label",example['text_label'])
inputs = tokenizer(example['text_label'], return_tensors="ms")

with mindspore._no_grad():
    outputs = model.generate(input_ids=inputs["input_ids"], max_new_tokens=10)
    print(tokenizer.batch_decode(outputs.asnumpy(), skip_special_tokens=True))

input `` Printed fabrics and related design expertise have always been the core of Marimekko 's business and brand .
true label neutral
['neutral']
